In [6]:
import deepchem as dc
import pandas as pd
import rdkit

_, tox21, _ =dc.molnet.load_tox21()

In [10]:
df = pd.DataFrame(tox21[0].ids, columns=['smiles'])



<DiskDataset X.shape: (6264, 1024), y.shape: (6264, 12), w.shape: (6264, 12), task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>